In [276]:
import pandas as pd
import pyarrow.parquet as pq
import numpy as np
import csv
import glob
from itertools import chain
from keras import models
from keras import layers
import os
import keras 
import sys


from keras.models import Sequential
from keras.layers import Dense


from collections import Counter
import matplotlib.pyplot as plt
from keras.callbacks import History

np.set_printoptions(threshold=sys.maxsize)

In [277]:
# given a path like "../trial/S01/ACC_01.parquet" return:
# the number of ms that a file needs in order to do the downsampling;
# and return the type among:
# - 0 (EDA)
# - 1 (BVP)
# - 2 (ACC)
# - 3 (TEM) 

# If the file is incorrect return -1
def type_file(name):
    name_file = name.split('/')[3][:3]
#     print(name_file)
    if name_file == 'EDA':
        return 0, 0
    elif name_file == 'BVP':
        return '250ms', 1
    elif name_file == 'ACC':
        return '250ms', 2
    elif name_file == 'TEM':
        return 0, 5
    return -1

In [278]:
# Function to determine the most frequent element in a list:
# [0,0,0,1,1] return 0
# [0,0,1,1,1] return 1

def most_frequent(List):
    counter = 0
    num = List[0]

    for i in List:
        curr_frequency = List.count(i)
        if(curr_frequency> counter):
            counter = curr_frequency
            num = i
    return num


# listt = [0,1,0,1]
# print(most_frequent(listt))

In [279]:
def append_data_and_fill_missing_zeros(ARRAY, df, value):
    for i in range(0, df.shape[0]):
        ARRAY.append(df[value][i])
#         print("BEFORE: ", len(ARRAY))

    if (len(ARRAY) % 2400) != 0:
        rounding = int(len(ARRAY)//2400 + (len(ARRAY) % 2400 > 0))
        number_cells = rounding * 2400
        missing_zeros = [0] * (number_cells - len(ARRAY))
        ARRAY = ARRAY + missing_zeros
    return ARRAY
    
    

In [280]:
def find_sleep_label(LABEL, EDA, df):
    i = 0
    list_sleep = []
#     create the list_sleep with all the values of the colums 'Sleep'
    for j in range(0, df.shape[0]):
        value = df['Sleep'][j].astype(np.float64)
        list_sleep.append(value)

#   for each 2400 window choose if 0 or 1, and append it to ARRAR_LABEL
    while i in range(0, len(list_sleep)- 2399):
        list_sleep_window = list_sleep[i:(i+2400)]
        label_sleep = most_frequent(list_sleep_window)
        LABEL.append(label_sleep)
        i += 2400
        
#     if EDA len is greather than ARRAY_LABEL, uniform it to the same length
    if (len(LABEL) < int(len(EDA)/2400)):
        missing_zeros = [0]*(int(len(EDA)/2400)-len(LABEL))
        LABEL = LABEL + missing_zeros
        
#     print(len(LABEL))
    return LABEL
        

In [281]:
def add_zeros_to_uniform_size_array(ARRAY, missing_zeros):
    ARRAY = [0]* missing_zeros
    return ARRAY

In [282]:
# Call function create_train_set_train_label so that we have the two zero tensors.
# for each file read the data, downsampling the data and fill the tensors.

def read_data(files):
    EDA, BVP, ACC_X, ACC_Y, ACC_Z, TEM, LABEL = [],[],[],[],[],[],[]

    
    # READ ALL THE FILES and DOWNSAMPLING
    for name_file in files:
#         print(name_file)
        rounding, number_cells, missing_zeros = 0, 0, []
        # DataFrame
        table = pd.read_parquet(name_file, engine='pyarrow')
        # creating DataFrame
        df = pd.DataFrame(table)
        # converting timestamp
        timestamp_col = pd.to_datetime(df['timestamp'], unit='s')
        df['timestamp'] = timestamp_col
        # get if file it's among EDA, BVP, ACC, ST, otherwise return error
        arr_type = type_file(name_file)
        if arr_type == -1:
            print("Error in the file name")
            return -1

        # RESAMPLE the tensor
        # 0 means it's already 4Hz, otherwise resample with '250ms'
        if arr_type[0] != 0:
            df = df.resample(arr_type[0], on='timestamp').mean()
            df = df.reset_index()
        
        
        print(df.shape)
       # CREATES THE ARRAYS
        # create linear arrays as sequences of elements. for each file add several zeros to complete a window of 2400     
        if arr_type[1] == 0:
            EDA = append_data_and_fill_missing_zeros(EDA, df, 'value')
            LABEL = find_sleep_label(LABEL, EDA, df)
                
        elif arr_type[1] == 1:
            BVP = append_data_and_fill_missing_zeros(BVP, df, 'value')

        elif arr_type[1] == 2:
            ACC_X = append_data_and_fill_missing_zeros(ACC_X, df, 'X')
            ACC_Y = append_data_and_fill_missing_zeros(ACC_Y, df, 'Y')
            ACC_Z = append_data_and_fill_missing_zeros(ACC_Z, df, 'Z')
            
        elif arr_type[1] == 5:
            TEM = append_data_and_fill_missing_zeros(TEM, df, 'value')
        
        
#     print("LEN LABEL",len(LABEL))
#     if each file has the same length it doesn make sense to check it. only in between files fill the row of zeros
#     and then start from the next row a new file
#     if the files are not the same quantity, it's better to throw error?
    uniform_len = max(len(EDA),len(BVP),len(ACC_X),len(ACC_Y),len(ACC_Z),len(TEM))    
#     print(len(EDA), ' ',len(BVP), ' ',len(ACC_X), ' ',len(ACC_Y), ' ',len(ACC_Z), ' ',len(TEM) )
#     check train_set
    if len(EDA) != uniform_len:
        EDA = EDA + [0]* (uniform_len - len(EDA))
    if len(BVP) != uniform_len:
        BVP = BVP + [0]* (uniform_len - len(BVP))
    if len(ACC_X) != uniform_len:
        ACC_X = ACC_X + [0]* (uniform_len - len(ACC_X))
    if len(ACC_Y) != uniform_len:
        ACC_Y = ACC_Y + [0]* (uniform_len - len(ACC_Y))
    if len(ACC_Z) != uniform_len:
        ACC_Z = ACC_Z + [0]* (uniform_len - len(ACC_Z))
    if len(TEM) != uniform_len:
        TEM = TEM + [0]* (uniform_len - len(TEM))
        

#     print(len(EDA_LABEL), ' ',len(BVP_LABEL), ' ',len(ACC_X_LABEL), ' ',len(ACC_Y_LABEL), ' ',len(ACC_Z_LABEL), ' ',len(TEM_LABEL) )
    # convert the list to a numpy array
    train_set = np.array([EDA, BVP, ACC_X, ACC_Y, ACC_Z, TEM],dtype=np.float64)
    train_label = np.array([LABEL], dtype=np.float64)
    

    # Reshape the tensor train set, counting the number of rows dividing by 2400
    row_training_set = uniform_len // 2400
    if (uniform_len % 2400) != 0:
        return "Error in missing_zeros"
    train_set = train_set.reshape(6,row_training_set,2400)
#     Reshape the tensor train label
#     train_label = train_label.reshape(6,uniform_len_label, 1)
    

    train_set = np.dstack(train_set)
    train_label = train_label.reshape(train_label.shape[1], 1)
    print(train_set.shape)
    print(train_label.shape)
    

    print("END read data")
    return train_set, train_label

In [283]:
def network_model(train_set, train_label) :
    model = Sequential()
    model.add(Dense(16, activation='relu', input_shape=(2400, 6), name="layer1"))
    model.add(Dense(12, activation='relu', name="layer2"))
    model.add(Dense(1, activation='sigmoid', name="layer3"))

    model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
    model.fit(np.array(train_set), np.array(train_label), epochs=15, batch_size=128)
    print("END network model")
    

In [296]:
def create_datasets(start_session, end_session):
#     create list of folders
    listt = os.listdir("../Sessions/")

#     list_dir = sorted(listt)[8:12]
    list_dir = sorted(listt)
    list_path_dir = []
    for elem in list_dir:
        list_path_dir.append("../Sessions/" + elem)

#     read files in the folder and create a list of files 
    list_ordered_files = []
    
    for folder in list_path_dir:
#         print(folder)
        list_files = []
        for infile in os.listdir(folder):
            list_files.append(infile)
        list_files = sorted(list_files)
#         print(list_files)

    #     divide the list into 4 sections: acc, bvp, eda, temp
    #     take one from each and construct a list 
    #     the order is really important while reading files
        
        acc = []
        bvp = []
        eda = []
        tem = []
        for elem in (list_files):
            if elem[:3] == "ACC":
                acc.append(elem)
            elif elem[:3] == "BVP":
                bvp.append(elem)
            elif elem[:3] == "EDA":
                eda.append(elem)
            elif elem[:3] == "TEM":
                tem.append(elem)

        acc = acc[start_session:end_session]
        bvp = bvp[start_session:end_session]
        eda = eda[start_session:end_session]
        tem = tem[start_session:end_session]
    #     create a list with that order: [acc1, bvp1, eda1, tem1, acc2, bvp2, eda2, tem2,...]
        for i in range(len(acc)):
            list_ordered_files.append(folder + "/" + acc[i])
            list_ordered_files.append(folder + "/" + bvp[i])
            list_ordered_files.append(folder + "/" + eda[i])
            list_ordered_files.append(folder + "/" + tem[i])
#     print(list_ordered_files)
        
    
    return read_data(list_ordered_files)

In [297]:
# 4 is the number of sessions per user
train_set, train_label = create_datasets(0,4)

(178064, 6)
(178064, 4)
(178064, 4)
(178064, 4)
(118536, 6)
(118536, 4)
(118536, 4)
(118536, 4)
(127292, 6)
(127292, 4)
(127292, 4)
(127292, 4)
(179620, 6)
(179620, 4)
(179620, 4)
(179620, 4)
(213868, 6)
(213868, 4)
(213868, 4)
(213868, 4)
(234096, 6)
(234096, 4)
(234096, 4)
(234096, 4)
(215920, 6)
(215920, 4)
(215920, 4)
(215920, 4)
(227504, 6)
(227504, 4)
(227504, 4)
(227504, 4)
(223204, 6)
(223204, 4)
(223204, 4)
(223204, 4)
(225564, 6)
(225564, 4)
(225564, 4)
(225564, 4)
(210372, 6)
(210372, 4)
(210372, 4)
(210372, 4)
(231104, 6)
(231104, 4)
(231104, 4)
(231104, 4)
(190148, 6)
(190148, 4)
(190148, 4)
(190148, 4)
(142856, 6)
(142856, 4)
(142856, 4)
(142856, 4)
(213716, 6)
(213716, 4)
(213716, 4)
(213716, 4)
(223320, 6)
(223320, 4)
(223320, 4)
(223320, 4)
(225860, 6)
(225860, 4)
(225860, 4)
(225860, 4)
(220156, 6)
(220156, 4)
(220156, 4)
(220156, 4)
(227520, 6)
(227520, 4)
(227520, 4)
(227520, 4)
(216128, 6)
(216128, 4)
(216128, 4)
(216128, 4)
(227188, 6)
(227188, 4)
(227188, 4)
(227

In [286]:
test_set, test_label = create_datasets(4,5)

(178064, 6)
(178064, 4)
(178064, 4)
(178064, 4)
(213868, 6)
(213868, 4)
(213868, 4)
(213868, 4)
(223204, 6)
(223204, 4)
(223204, 4)
(223204, 4)
(190148, 6)
(190148, 4)
(190148, 4)
(190148, 4)
(225860, 6)
(225860, 4)
(225860, 4)
(225860, 4)
(227188, 6)
(227188, 4)
(227188, 4)
(227188, 4)
(145364, 6)
(145364, 4)
(145364, 4)
(145364, 4)
(235188, 6)
(235188, 4)
(235188, 4)
(235188, 4)
(189588, 6)
(189588, 4)
(189588, 4)
(189588, 4)
(206336, 6)
(206336, 4)
(206336, 4)
(206336, 4)
(180316, 6)
(180316, 4)
(180316, 4)
(180316, 4)
(212684, 6)
(212684, 4)
(212684, 4)
(212684, 4)
(205104, 6)
(205104, 4)
(205104, 4)
(205104, 4)
(229924, 6)
(229924, 4)
(229924, 4)
(229924, 4)
(217200, 6)
(217200, 4)
(217200, 4)
(217200, 4)
(225776, 6)
(225776, 4)
(225776, 4)
(225776, 4)
(1386, 2400, 6)
(1386, 1)
END read data


In [298]:
network_model(train_set,train_label)

Epoch 1/15
43/43 [==============================] - 1s 4ms/step - loss: nan - accuracy: 0.5710
Epoch 2/15
43/43 [==============================] - 0s 4ms/step - loss: nan - accuracy: 0.5715
Epoch 3/15
43/43 [==============================] - 0s 4ms/step - loss: nan - accuracy: 0.5711
Epoch 4/15
43/43 [==============================] - 0s 4ms/step - loss: nan - accuracy: 0.5813
Epoch 5/15
43/43 [==============================] - 0s 4ms/step - loss: nan - accuracy: 0.5782
Epoch 6/15
43/43 [==============================] - 0s 4ms/step - loss: nan - accuracy: 0.5692
Epoch 7/15
43/43 [==============================] - 0s 4ms/step - loss: nan - accuracy: 0.5759
Epoch 8/15
43/43 [==============================] - 0s 4ms/step - loss: nan - accuracy: 0.5798
Epoch 9/15
43/43 [==============================] - 0s 4ms/step - loss: nan - accuracy: 0.5674
Epoch 10/15
43/43 [==============================] - 0s 4ms/step - loss: nan - accuracy: 0.5726
Epoch 11/15
43/43 [==============================